In [77]:
# %load Resolve.py
# Resolve.py
"""
We consider a normal order ramified on snc but no secondary ramification. We
blow up repeatedly at nodes recording the log discrepancy and
b-discrepancy until we have all possible negative discrepancy curves.

The exceptional curves should probably be indexed by dyadic fractions, written
as binary numbers, but we will instead clear denominators and work with
integers. 
"""

from fractions import Fraction


# Enter initial ramification data.
# The ramification cover of the curve i is gi copies of zi modulo n
#initial_ram_data = input("Please enter the ramification data tuple n,z1,z2,g1,g2\n")
#n_str, z1_str, z2_str, g1_str, g2_str = initial_ram_data.split(',')
#n = int(n_str.strip())
#z1 = int(z1_str.strip())
#z2 = int(z2_str.strip())
#g1 = int(g1_str.strip())
#g2 = int(g2_str.strip())


#This function computes the order of an element in a cyclic group.
def order(num,modulus):
    ord = 1
    while ord*num %modulus !=0:
        ord +=1
    return ord


#print("Corresponding maximal order has ram data", end=':')
#print(z1 % n, z2 % n, ' modulo ', n, end='')
#f1 = order(z1,n) # these are ramification indices of max order
#f2 = order(z2,n)
#print(' giving ramification indices ', f1, f2,'.')
# Check working by showing ramification of containing maximal order



"""
This is the function that computes log and b-discrepancies of
all exceptional curves in a good log resolution. Here the good log
resolution is one chosen so all negative discrepancy curves are
guaranteed to be found there. The data will be recorded in a dictionary.
"""
def resolve(n,z1,z2,g1,g2):
    f1 = order(z1,n) # these are ramification indices of max order
    f2 = order(z2,n)
    no_blowups = max(g1,g2)*n -1
    # crude bound on number of blowups to ensure good log resolution
    print(f'We need to blowup set of nodes  at most {no_blowups} times.')
    no_curves = 2**no_blowups # number of exc curves actually 2**no_blowups -1
    curves = {0:[Fraction(1,f1*g1)-1,z1] , no_curves:[Fraction(1,f2*g2)-1,z2]}
    # Create dictionary of curve data. Keys are 0 up to no_curves.
    # Current values are lists of [log discrepancy, ram of max order] 
    step = no_curves # Final indices for strict transforms of initial ram 
                     # curves are `step' apart

    
    for i in range(1,no_blowups+1):

        print(f'This is the {i}-th blowup. New curves have  log and b- discrepancy and ramification:')
        l = len(curves)-1 # the number of new exceptional curves
       
        oldcurves = set(curves.keys()).difference({0,no_curves})
        for index in oldcurves:
            curves[index] = [curves[index][0], curves[index][1], curves[index][2],curves[index][3]-2]                                                         
        step = int(step/2) # indices of i-th blowup curves have indices 'step' apart
        
        is_good_res = True
        # We'll use this to test if the i-th blowup gives a good resolution

        for j in range(l):
            log_disc = 1 + curves[2*j*step][0] + curves[(2*j+2)*step][0]
            #  Give log discrepancy of j-th new exceptional in i-th blowup
            is_good_res =  is_good_res and (log_disc >=0)
            ram = (curves[2*j*step][1] + curves[(2*j+2)*step][1])%n
            # Gives ramification along j-th new exceptional in i-th blowup
            b_disc = log_disc + 1-Fraction(1,order(ram,n))
            # Compute the b-discrepancy
            curves[(2*j +1)*step] = [log_disc, ram, b_disc,-1]
            # Update curves dictionary with entry for j-th curve in i-th blowup
            print(log_disc,b_disc, ram, end=';')

        if is_good_res:
            break
        else:
            print(' DONE', end='\n\n')
    return curves
    print('\n\n Good log resolution has been achieved.')

def continuedFraction(a):
    return a
    
def contract(curves):
    indices = sorted(curves)
    indices.pop()
    del indices[0]
    last = max(indices)
    first = min(indices)
    tocontract = [ i for i in indices if curves[i][2] >= 0 ]
    # next to contract only -1 curves
    print(tocontract)
    print(curves)
    for i in tocontract:
        if curves[i][3] == -1:
            curves[min(i+1,last)][3] += 1
            curves[max(i-1,first)][3] += 1
            curves.pop(i)
    print(sorted(curves))        
    return curves

    
#resolve(n,z1,z2,g1,g2)


In [78]:
cv = resolve(5,3,1,1,1)

We need to blowup set of nodes  at most 4 times.
This is the 1-th blowup. New curves have  log and b- discrepancy and ramification:
-3/5 1/5 4; DONE

This is the 2-th blowup. New curves have  log and b- discrepancy and ramification:
-2/5 2/5 2;-2/5 -2/5 0; DONE

This is the 3-th blowup. New curves have  log and b- discrepancy and ramification:
-1/5 -1/5 0;0 4/5 1;0 4/5 4;-1/5 3/5 1; DONE

This is the 4-th blowup. New curves have  log and b- discrepancy and ramification:
0 4/5 3;2/5 6/5 2;3/5 7/5 3;2/5 2/5 0;2/5 6/5 3;3/5 7/5 4;2/5 6/5 1;0 4/5 2;

In [79]:
cv

{0: [Fraction(-4, 5), 3],
 16: [Fraction(-4, 5), 1],
 8: [Fraction(-3, 5), 4, Fraction(1, 5), -7],
 4: [Fraction(-2, 5), 2, Fraction(2, 5), -5],
 12: [Fraction(-2, 5), 0, Fraction(-2, 5), -5],
 2: [Fraction(-1, 5), 0, Fraction(-1, 5), -3],
 6: [Fraction(0, 1), 1, Fraction(4, 5), -3],
 10: [Fraction(0, 1), 4, Fraction(4, 5), -3],
 14: [Fraction(-1, 5), 1, Fraction(3, 5), -3],
 1: [Fraction(0, 1), 3, Fraction(4, 5), -1],
 3: [Fraction(2, 5), 2, Fraction(6, 5), -1],
 5: [Fraction(3, 5), 3, Fraction(7, 5), -1],
 7: [Fraction(2, 5), 0, Fraction(2, 5), -1],
 9: [Fraction(2, 5), 3, Fraction(6, 5), -1],
 11: [Fraction(3, 5), 4, Fraction(7, 5), -1],
 13: [Fraction(2, 5), 1, Fraction(6, 5), -1],
 15: [Fraction(0, 1), 2, Fraction(4, 5), -1]}

In [80]:
contract(cv)

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15]
{0: [Fraction(-4, 5), 3], 16: [Fraction(-4, 5), 1], 8: [Fraction(-3, 5), 4, Fraction(1, 5), -7], 4: [Fraction(-2, 5), 2, Fraction(2, 5), -5], 12: [Fraction(-2, 5), 0, Fraction(-2, 5), -5], 2: [Fraction(-1, 5), 0, Fraction(-1, 5), -3], 6: [Fraction(0, 1), 1, Fraction(4, 5), -3], 10: [Fraction(0, 1), 4, Fraction(4, 5), -3], 14: [Fraction(-1, 5), 1, Fraction(3, 5), -3], 1: [Fraction(0, 1), 3, Fraction(4, 5), -1], 3: [Fraction(2, 5), 2, Fraction(6, 5), -1], 5: [Fraction(3, 5), 3, Fraction(7, 5), -1], 7: [Fraction(2, 5), 0, Fraction(2, 5), -1], 9: [Fraction(2, 5), 3, Fraction(6, 5), -1], 11: [Fraction(3, 5), 4, Fraction(7, 5), -1], 13: [Fraction(2, 5), 1, Fraction(6, 5), -1], 15: [Fraction(0, 1), 2, Fraction(4, 5), -1]}
[0, 2, 4, 6, 8, 10, 12, 14, 16]


{0: [Fraction(-4, 5), 3],
 16: [Fraction(-4, 5), 1],
 8: [Fraction(-3, 5), 4, Fraction(1, 5), -5],
 4: [Fraction(-2, 5), 2, Fraction(2, 5), -3],
 12: [Fraction(-2, 5), 0, Fraction(-2, 5), -3],
 2: [Fraction(-1, 5), 0, Fraction(-1, 5), -1],
 6: [Fraction(0, 1), 1, Fraction(4, 5), -1],
 10: [Fraction(0, 1), 4, Fraction(4, 5), -1],
 14: [Fraction(-1, 5), 1, Fraction(3, 5), -1]}